In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from pprint import pprint
import googlemaps
import json
import os
import requests

In [2]:
API_KEY= os.getenv('GOOGLE_MAP_API_KEY')
gmap = googlemaps.Client(API_KEY)

In [ ]:
hotels = []

# Open the file and read it line by line
with open('Datasets/Hotels.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        json_obj = json.loads(line.strip())  # strip() removes any leading/trailing whitespace
        hotels.append(json_obj)

In [4]:
pprint(hotels[10])

{'address': '1200 Market St',
 'attributes': {'Alcohol': "u'full_bar'",
                'BusinessAcceptsCreditCards': 'True',
                'BusinessParking': "{'garage': False, 'street': False, "
                                   "'validated': False, 'lot': False, 'valet': "
                                   'True}',
                'Caters': 'True',
                'DogsAllowed': 'True',
                'GoodForKids': 'True',
                'HasTV': 'True',
                'NoiseLevel': "u'average'",
                'OutdoorSeating': 'False',
                'RestaurantsAttire': "u'casual'",
                'RestaurantsDelivery': 'False',
                'RestaurantsPriceRange2': '3',
                'RestaurantsReservations': 'True',
                'WiFi': "u'paid'"},
 'business_id': 'D5V0Fawd6ODVgqCY8xngsw',
 'categories': 'Event Planning & Services, Hotels & Travel, Restaurants, '
               'Hotels, Seafood',
 'city': 'Philadelphia',
 'hours': None,
 'is_open': 1,
 'lat

In [92]:
#a working example
name = 'Best Western Center City' #name + address + city
address = '501 N 22nd St'
city = 'Philadelphia'

query = f'{name}, {address}, {city}'
print(query)


url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?fields=formatted_address%2Cname%2Cplace_id&input={query}&inputtype=textquery&key={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)

Best Western Center City, 501 N 22nd St, Philadelphia
{'candidates': [{'formatted_address': '501 N 22nd St, Philadelphia, PA 19130, United States', 'name': 'Best Western Center City Hotel', 'place_id': 'ChIJXcugoMvHxokRTfET8k8ACXo'}], 'status': 'OK'}


In [98]:
#try on a business that is closed but shown as still open on yelp (yelp dataset is old!) already
query = 'Rodeway Inn Center City, 1208 Walnut St, Philadelphia' #name + address + city

url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?fields=name%2Cformatted_address%2Cplace_id&input={query}&inputtype=textquery&key={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)

#it will return us the current business and the one before. We can just take the last one. 

{'candidates': [{'formatted_address': '1208 Walnut St, Philadelphia, PA 19107, United States', 'name': 'Quality Inn - Center City', 'place_id': 'ChIJn3S9ub3HxokR5697EmTqp5U'}, {'formatted_address': '1208 Walnut St, Philadelphia, PA 19107, United States', 'name': 'Rodeway Inn Center City', 'place_id': 'ChIJPRoD-CXGxokR3XOibY2ErZc'}], 'status': 'OK'}


In [3]:

#try on a business that is closed already on yelp dataset
query = 'Skyview Plaza Hotel, 2033 Penrose Ave, Philadelphia' #name + address + city

url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?fields=name%2Cformatted_address%2Cplace_id&input={query}&inputtype=textquery&key={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)

#it will return us random stuff, so we need to get rid of all is_open = 0 businesses

{'candidates': [{'formatted_address': '2033 Penrose Ave, Philadelphia, PA 19145, United States', 'name': 'Holiday Inn Philadelphia Arpt-Stadium Area, an IHG Hotel', 'place_id': 'ChIJ8QkIp9zFxokRq5byqpgOLeg'}, {'formatted_address': '2033 Penrose Ave, Philadelphia, PA 19145, United States', 'name': 'Radisson Hotel Philadelphia, PA', 'place_id': 'ChIJtaufYTHFxokR7RKXTt1Ysi4'}], 'status': 'OK'}


In [5]:
#Try on business shown as open on yelp but closed on google 
place_id = 'ChIJPRoD-CXGxokR3XOibY2ErZc'
response = gmap.place(place_id = place_id)

response


# if the id of a permantly closed business is passed into place, it won;t show the price_level, but the permantly closed attribute is true. So we can filter out them.

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '1208',
    'short_name': '1208',
    'types': ['street_number']},
   {'long_name': 'Walnut Street',
    'short_name': 'Walnut St',
    'types': ['route']},
   {'long_name': 'Washington Square West',
    'short_name': 'Washington Square West',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Philadelphia',
    'short_name': 'Philadelphia',
    'types': ['locality', 'political']},
   {'long_name': 'Philadelphia County',
    'short_name': 'Philadelphia County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Pennsylvania',
    'short_name': 'PA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '19107', 'short_name': '19107', 'types': ['postal_code']}],
  'adr_address': '<span class="street-address">1208 Walnut St</span>, <span class="locality">Ph

In [6]:
#Try on business shown as open everywhere
place_id = 'ChIJXcugoMvHxokRTfET8k8ACXo'
response = gmap.place(place_id = place_id)

response


# if the id of a permantly closed business is passed into place, it won;t show the price_level, but the permantly closed attribute is true. So we can filter out them.

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '501',
    'short_name': '501',
    'types': ['street_number']},
   {'long_name': 'North 22nd Street',
    'short_name': 'N 22nd St',
    'types': ['route']},
   {'long_name': 'Center City',
    'short_name': 'Center City',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Philadelphia',
    'short_name': 'Philadelphia',
    'types': ['locality', 'political']},
   {'long_name': 'Philadelphia County',
    'short_name': 'Philadelphia County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Pennsylvania',
    'short_name': 'PA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '19130', 'short_name': '19130', 'types': ['postal_code']},
   {'long_name': '3507',
    'short_name': '3507',
    'types': ['postal_code_suffix']}],
  'adr_address': '<span cl

In [11]:
#use an open business and see the result from places api (new)
place_id = 'ChIJXcugoMvHxokRTfET8k8ACXo'

return_fileds = 'pricelevel'

url = f'https://places.googleapis.com/v1/places/{place_id}?fields={return_fileds}&key={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)

response
#I tried on google sandbox and it says this particular business has no price level attributes

Error: 400


<Response [400]>

In [13]:

#use an open business and see the result from places api (old)
place_id = 'ChIJXcugoMvHxokRTfET8k8ACXo'

return_fileds = 'name,price_level,rating'

url = f'https://maps.googleapis.com/maps/api/place/details/json?fields={return_fileds}&place_id={place_id}&key={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)

response
#the result of both new and old place api tells us that it's not the problem of the api, but this location itself has no price level attributes.



{'html_attributions': [], 'result': {'name': 'Best Western Center City Hotel', 'rating': 3.3}, 'status': 'OK'}


<Response [200]>

In [14]:

#use an open business that I know has price level for sure and see the result from places api (old)
#Shedd Aquarium
place_id = 'ChIJ-XW3X2MrDogR3_tQ3-OdBTI'

return_fileds = 'name,price_level,rating'

url = f'https://maps.googleapis.com/maps/api/place/details/json?fields={return_fileds}&place_id={place_id}&key={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)

response
#the result of both new and old place api tells us that it's not the problem of the api, but this location itself has no price level attributes.

{'html_attributions': [], 'result': {'name': 'Shedd Aquarium', 'price_level': 3, 'rating': 4.5}, 'status': 'OK'}


<Response [200]>

In [15]:
#to see if I can live with it, I have to determine if no price level means lack of reviews or it's free.
#in local flavor, we have a rocky statue, let's explore it
#get business id
name = 'The Rocky Statue' #name + address + city
address = '2600 Benjamin Franklin Pkwy'
city = 'Philadelphia'

query = f'{name}, {address}, {city}'


url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?fields=formatted_address%2Cname%2Cplace_id&input={query}&inputtype=textquery&key={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)

#the id is: ChIJd4XRlLXHxokR24097rPRpx0

The Rocky Statue, 2600 Benjamin Franklin Pkwy, Philadelphia
{'candidates': [{'formatted_address': '2600 Benjamin Franklin Pkwy, Philadelphia, PA 19130, United States', 'name': 'Rocky Statue', 'place_id': 'ChIJd4XRlLXHxokR24097rPRpx0'}, {'formatted_address': '2600 Benjamin Franklin Pkwy, Philadelphia, PA 19130, United States', 'name': 'Philadelphia Museum of Art Steps', 'place_id': 'ChIJ33ZAsLXHxokRRidoQ-ctnM4'}], 'status': 'OK'}


In [16]:
#rocky status, see if it has price level
place_id = 'ChIJd4XRlLXHxokR24097rPRpx0'

return_fileds = 'name,price_level,rating'

url = f'https://maps.googleapis.com/maps/api/place/details/json?fields={return_fileds}&place_id={place_id}&key={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)

response
#no price level.

{'html_attributions': [], 'result': {'name': 'Rocky Statue', 'rating': 4.7}, 'status': 'OK'}


<Response [200]>

In [17]:

#return_fileds = '*'

url = f'https://places.googleapis.com/v1/places/{place_id}?fields=*&key={API_KEY}'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)


{'name': 'places/ChIJd4XRlLXHxokR24097rPRpx0', 'id': 'ChIJd4XRlLXHxokR24097rPRpx0', 'types': ['tourist_attraction', 'point_of_interest', 'establishment'], 'formattedAddress': '2600 Benjamin Franklin Pkwy, Philadelphia, PA 19130, USA', 'addressComponents': [{'longText': '2600', 'shortText': '2600', 'types': ['street_number'], 'languageCode': 'en-US'}, {'longText': 'Benjamin Franklin Parkway', 'shortText': 'Benjamin Franklin Pkwy', 'types': ['route'], 'languageCode': 'en'}, {'longText': 'West Philadelphia', 'shortText': 'West Philadelphia', 'types': ['neighborhood', 'political'], 'languageCode': 'en'}, {'longText': 'Philadelphia', 'shortText': 'Philadelphia', 'types': ['locality', 'political'], 'languageCode': 'en'}, {'longText': 'Philadelphia County', 'shortText': 'Philadelphia County', 'types': ['administrative_area_level_2', 'political'], 'languageCode': 'en'}, {'longText': 'Pennsylvania', 'shortText': 'PA', 'types': ['administrative_area_level_1', 'political'], 'languageCode': 'en'},